In [6]:
!pip install -q evaluate rouge-score FlagEmbedding pandas peft scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 9.1 MB/s eta 0:00:00


In [2]:
# R2C eligibility docs
DOC_URL = 'https://secure.ssa.gov/apps10/poms.nsf/lnx/0500502100'

In [ ]:
!pip install -q nest_asyncio
import nest_asyncio
nest_asyncio.apply()
import os
os.environ['ANTHROPIC_API_KEY'] = 'sk-ant-apikey-copy-to-here'

# Test cases

> I assume you are using Linux machine with `wget` installed

In [9]:
!wget https://raw.githubusercontent.com/Georgetown-MDI-Beeck-Collaboration/p2c2024-RAG-benchmark/SSI_SSDI-result/SSI-SSDI/scenario-test-cases.jsonl

--2024-09-10 00:01:12--  https://raw.githubusercontent.com/Georgetown-MDI-Beeck-Collaboration/p2c2024-RAG-benchmark/SSI_SSDI-result/SSI-SSDI/scenario-test-cases.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5546 (5.4K) [text/plain]
Saving to: ‘scenario-test-cases.jsonl’

scenario-test-cases 100%[===================>]   5.42K  --.-KB/s    in 0s      

2024-09-10 00:01:12 (45.6 MB/s) - ‘scenario-test-cases.jsonl’ saved [5546/5546]



In [3]:
import json
import pandas as pd
data = pd.read_json('scenario-test-cases.jsonl', lines=True)
questions = data["question"].tolist()
references = data["answer"].tolist()

In [4]:
import evaluate
rouge = evaluate.load('rouge')

In [7]:
import numpy as np
from FlagEmbedding import BGEM3FlagModel

model = BGEM3FlagModel('BAAI/bge-m3',
                       use_fp16=True)

embeddings_1 = model.encode(references)['dense_vecs']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

...

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

sparse_linear.pt:   0%|          | 0.00/3.52k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/FlagEmbedding/BGE_M3/modeling.py:335: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  colbert_state_dict = torch.load(os.path.join(mode

## Manual input your record

In [30]:
predictions = [
    """Based on the provided context, you may be eligible for Supplemental Security Income (SSI) as an asylee. Asylees are considered "qualified aliens" for SSI eligibility and are exempt from the 5-year bar that applies to most qualified aliens who entered the U.S. on or after August 22, 1996 (Alien0050 lines 0-39). Asylees may be eligible for SSI for up to 7 years from the date of obtaining that status, and you have been in the U.S. for 5 years, which is within this eligibility window (Alien0050 lines 82-122). Being too sick to work may qualify as a disability, which is a factor in SSI eligibility for certain aliens (Alien0050 lines 82-122). However, to be eligible for SSI, you must also meet other criteria, including income and resource limits (Alien0050 lines 0-39).
    """
    ,
    """Based on the provided context, the situation is complex and not fully addressed in the given information. Veterans and active duty members of the U.S. Armed Forces are generally eligible for SSI (Alien0050 lines 39-82, Basic2023 pages 5-8). However, qualified aliens eligible based on veteran status can lose eligibility if discharged under certain conditions (Alien0050 lines 82-122). The context does not specifically address dishonorable discharge or PTSD.

    Alternatively, you may be eligible if you are ""blind or disabled and was lawfully residing in the U.S. on 8/22/96"" (Basic2023 pages 19-22). However, the context does not provide details on how disability is determined for SSI eligibility.

    Given the complexity of your situation, it is advisable to consult with a Social Security Administration representative for a definitive answer regarding your SSI eligibility.
    """
    ,
    """Based on the provided context, it appears that your SSI eligibility may be affected by your husband's passing, but the situation is complex and depends on several factors. As a spouse of a veteran, your eligibility for SSI could potentially end with the month after your husband's death (Basic2023 pages 9-11). However, qualified aliens, including spouses of veterans, may retain eligibility under certain circumstances (Alien0050 lines 0-39). Your continued eligibility will depend on meeting both non-citizen requirements and other SSI eligibility criteria, such as income and resource limits (Alien0050 lines 0-39). It's important to note that changes in immigration status or other circumstances may necessitate a redetermination of your eligibility (Basic2023 pages 8-9).
    """
    ,
    """As a refugee granted status about six and a half years ago, your SSI eligibility is subject to a 7-year time limitation from the date your refugee status was granted (Basic2023 pages 5-8, Alien0050 lines 39-82). This means your current eligibility may end in about six months. However, since you are on dialysis, which likely qualifies you as disabled, you may be eligible for an exception to this time limit. If you were lawfully residing in the United States on August 22, 1996, and are considered blind or disabled, you could continue to receive SSI benefits beyond the 7-year period (Basic2023 pages 16-19, Alien0050 lines 0-39). It's advisable to contact the Social Security Administration to confirm your ongoing eligibility and explore potential exceptions.
    """
    ,
    """Based on the provided context, it appears unlikely that you would be eligible for SSI benefits in your current situation. To be eligible for SSI, an individual must be either a U.S. citizen or a ""qualified alien"" (Alien0050 lines 0-39). Without official immigration papers, you would likely be considered a ""nonqualified alien,"" which generally precludes SSI eligibility unless grandfathered in from before August 22, 1996 (Alien0050 lines 82-122). The Social Security Administration requires verification of alien status for eligibility determination (Alien0050 lines 82-122). While there are some exceptions for certain categories of immigrants, such as refugees or asylees, these do not appear to apply to your situation as described (Basic2023 pages 1-3, Basic2023 pages 13-16).
    """
    ,
    """As a lawful permanent resident (Green Card holder), you may be eligible for Supplemental Security Income (SSI) based on your work history and current health status. To qualify, you generally need 40 qualifying quarters of work (Basic2023 pages 19-22). However, since you entered the U.S. before August 22, 1996, and are experiencing health issues that may qualify as a disability, you might be eligible even without meeting the 40-quarter requirement (Basic2023 pages 13-16). Your receipt of other public assistance may affect your eligibility (Alien0050 lines 82-122). To determine your eligibility, you must meet both non-citizen requirements and all other SSI eligibility criteria, including income and resource limits (Basic2023 pages 1-3). A formal evaluation of your specific situation is necessary to confirm eligibility.
    """
    ,
    """As a refugee granted status two years ago, you are currently eligible for SSI due to your disability. However, your eligibility is subject to a 7-year time limitation from the date your refugee status was granted (Alien0050 lines 39-82, Basic2023 pages 5-8). This means you will remain eligible for approximately five more years, assuming no changes in your status or other eligibility factors (Basic2023 pages 16-19). After the 7-year period, your eligibility may be reevaluated. If you continue to meet other SSI eligibility requirements, such as being blind or disabled, you may be able to maintain your benefits beyond the 7-year limit (Basic2023 pages 5-8, Alien0050 lines 0-39).
    """
    ,
    """As a veteran, you may be eligible to apply for Supplemental Security Income (SSI). Veterans are considered ""qualified aliens"" for SSI eligibility purposes and are exempt from the 5-year bar that applies to most qualified aliens who entered the U.S. on or after August 22, 1996 (Alien0050 lines 0-39). Your status as a veteran potentially qualifies you for SSI benefits, regardless of other conditions typically applied to immigrants (Basic2023 pages 16-19). However, eligibility is not guaranteed solely based on veteran status. You must also meet other SSI eligibility criteria, such as being aged, blind, or disabled, and having income and resources within allowable limits (Alien0050 lines 0-39). Your PTSD may be considered in determining disability status.
    """
    ,
    """Yes, remarriage would impact your SSI eligibility. As the spouse of a veteran, your eligibility for SSI ends with the month after the month of remarriage (Basic2023 pages 9-11). Your current eligibility is based on your status as the spouse of a veteran or active duty military member, which is an exception to the 5-year bar for qualified aliens (Alien0050 lines 0-39). Remarriage would change your status and likely result in the loss of SSI eligibility (Alien0050 lines 82-122). It's important to note that qualified alien status alone does not guarantee SSI eligibility, and changes in marital status can affect eligibility for those whose status is based on a relationship to a veteran (Basic2023 pages 8-9, Basic2023 pages 19-22).
    """
    ,
    """Based on the provided context, your eligibility for SSI as an asylee who arrived 10 years ago is uncertain. Asylees are considered ""qualified aliens"" for SSI eligibility purposes (Basic2023 pages 1-3). However, there is a 7-year time limit on SSI eligibility for asylees from the date of obtaining that status (Alien0050 lines 82-122). Since you arrived 10 years ago, you may have exceeded this time limit. To be eligible for SSI, you must meet both non-citizen requirements and other SSI eligibility factors, including being aged, blind, or disabled, and having income and resources within allowable limits (Alien0050 lines 0-39). Given your age and health condition, you may meet some of these criteria, but your asylee status duration could be a limiting factor.
    """
]

In [23]:
results = rouge.compute(predictions=predictions,
                        references=references)
print(results)

{'rouge1': 0.09550044274878426, 'rouge2': 0.018672250657445558, 'rougeL': 0.06790505095745589, 'rougeLsum': 0.08147286797283657}


In [31]:
embeddings_2 = model.encode(predictions)['dense_vecs']
similarity = embeddings_1 @ embeddings_2.T
dist = [similarity[i][i] for i in range(len(similarity))]
print('cosine similarity (mean, std)',
      np.mean(dist), np.std(dist))

cosine similarity (mean, std) 0.76718175 0.07441664


## Automated testing
We picked the implementation of paper-qa as our demo

In [34]:
!pip install paper-qa sentence-transformers markdownify

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
  Created wheel for html2text: filename=html2text-2024.2.26-py3-none-any.whl size=33111 sha256=599939a07230cb7fe97fddd3e557cdd19512db836fd29c55ddabcdc09e0f7789
  Stored i

In [35]:
import requests
import markdownify

def download_html_to_markdown(url: str) -> str:
    """
    Downloads the HTML content from a given URL and converts it into a Markdown string.

    Args:
        url (str): The URL of the web page to download and convert.

    Returns:
        str: The Markdown string representation of the downloaded HTML content.
    """
    # Send a GET request to the URL
    response = requests.get(url)
    response.raise_for_status()  # Raise an HTTPError if the response status code is 4XX/5XX

    # Convert the HTML content to Markdown using markdownify
    markdown_content = markdownify.markdownify(response.text, heading_style="ATX")

    return markdown_content

In [ ]:
from paperqa import Docs, SentenceTransformerEmbeddingModel
from langchain_anthropic import ChatAnthropic

MODEL_NAME = "claude-3-5-sonnet-20240620"
class MyChatAnthropic(ChatAnthropic):
    model_name = MODEL_NAME
llm = MyChatAnthropic(model=MODEL_NAME,
                    temperature=0.2,
                    max_tokens=4096,)

class MyEmb(SentenceTransformerEmbeddingModel):
    async def aembed_documents(self, texts):
        return await self.embed_documents(None, texts)
emb = MyEmb(
    model_name="mixedbread-ai/mxbai-embed-large-v1")

docs = Docs(llm="langchain",
            embedding="langchain",
            embedding_client=emb,
            client=llm)
docs.max_concurrent = 1

In [ ]:
with open('doc.md', 'w') as f:
    doc = download_html_to_markdown(DOC_URL)
    f.write(doc)

docs.add('doc.md', disable_check=True)

In [ ]:
predictions = [docs.query(question).answer for question in questions]

In [ ]:
results = rouge.compute(predictions=predictions,
                        references=references)
print(results)

In [ ]:
embeddings_2 = model.encode(predictions)['dense_vecs']
similarity = embeddings_1 * embeddings_2.T
print('cosine similarity (median)', np.median(similarity))

# Test cases

In [32]:
!wget https://raw.githubusercontent.com/Georgetown-MDI-Beeck-Collaboration/p2c2024-RAG-benchmark/SSI_SSDI-result/SSI-SSDI/test-cases.jsonl

--2024-09-10 00:37:55--  https://raw.githubusercontent.com/Georgetown-MDI-Beeck-Collaboration/p2c2024-RAG-benchmark/SSI_SSDI-result/SSI-SSDI/test-cases.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6871 (6.7K) [text/plain]
Saving to: ‘test-cases.jsonl’

test-cases.jsonl    100%[===================>]   6.71K  --.-KB/s    in 0s      

2024-09-10 00:37:55 (44.2 MB/s) - ‘test-cases.jsonl’ saved [6871/6871]



In [33]:
import json
import pandas as pd
data = pd.read_json('test-cases.jsonl', lines=True)
questions = data["question"].tolist()
references = data["answer"].tolist()

In [42]:
predictions = [
    True,
    True,
    True,
    True,
    True,
    False,
    False,
    True,
    True,
    False,
    True,
    True,
    True,
    True,
    True,
    True,
    True,
    True,
    True,
    True,
    True,
    True,
    True,
    True,
    True,
    True,
    True,
    True,
    False,
    True,
    True,
    False,
    False,
    False,
    False,
    False,
    False,
    False,
    False,
    False,
    False,
    False,
    False,
    False,
    False,
    False,
    False,
    False,
    True,
    False,
    True,
    False,
    True,
    True,
    False,
    False,
    False,
    False,
]

In [52]:
from sklearn.metrics import f1_score
print(f1_score(references, predictions))

0.878048780487805
